<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2022
Prof. Dr. Heiner Giefers

# Das Perzeptron

In diesem Arbeitsblatt wollen wir ein Modell für das Perzeptron entwickeln.
Um unsere Implementierung zu testen, wollen wir einen realistischen Datensatz verwenden.
Hier bietet sich der Schwertlilien (*Iris*) Datensatz an.
Dieser beinhaltet 150 Datenpunkte und drei Klassen, wobei jede Lilienart mit genau 50 Repräsentanten vertreten ist.
Die Klassen lauten:
- **0** = Iris-Setosa (Borsten-Schwertlilie)
- **1** = Iris-Versicolour (Verschiedenfarbige Schwertlilie)
- **2** = Iris-Virginica (Verschiedenfarbige Schwertlilie)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
iris = datasets.load_iris()

df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
    columns= ["Kelchblatt Laenge", "Kelchblatt Breite", "Kronblatt Laenge", "Kronblatt Breite"] + ['target'])

df

## Perzeptron Operationen

Bevor wir uns dem Datensatz widmen, wollen wir ein Minimalbeispiel "per Hand" durchspielen.

In [ ]:
# Inititalisieren der Parameter b und x
b = np.array([.0])
w = np.array([.0, .0])

# X besteht aus 4 Punkten und 2 Attributen
X = np.array([[10,1],[20,2],[40,6],[50,5]])

# Die Label y
y = np.array([0,0,1,1])

# Die Lernregel lautet
def update(x, y, w, b, eta=0.5):
    # 1. Berechne die Uebertragungsfunktion
    z = w@x.T + b
    # 2. Berechne die Aktivierungsfunktion
    a = (z>0.0)*1.0
    # 3. Berechne den Fehler für den aktuellen Datenpunkt
    #    und multipliziere ihn mit der Lernrate. Es gilt:
    #    - Ausgabe ist 0, Label ist 1 => inkrementiere die Gewichte
    #    - Ausgabe ist 1, Label ist 0 => dekrementiere die Gewichte
    d = eta * (y-a)
    # 4. Korrigiere den Bias Parameter mit dem Fehler-Term
    b += d
    # 5. Korrigiere den Bias Parameter mit dem Fehler-Term
    w += d * x
    

for i in range(len(X)):
    update(X[i], y[i], w, b, eta=0.1)
    print(w, b)

Wir lassen das Perzeptron nun 100 *Epochen* lernen:

In [ ]:
for k in range(100):
    for i in range(len(X)):
        update(X[i], y[i], w, b, eta=0.1)

print("Die Parameter lauten:", w, b)

Welche Vorhersage liefern die Parameter für unsere Daten?

In [ ]:
z = w@X.T + b
a = (z>0.0)*1.0
a

Wir können die Punkte und die Entscheidungsgrenze nun visualisieren:

In [ ]:
a = - w[0] / w[1]
xx = np.linspace(10, 50)
yy = a * xx - (b[0]) / w[1]
plt.plot(xx,yy)
plt.scatter(X[y==0,0],X[y==0,1], c='b', marker='x')
plt.scatter(X[y==1,0],X[y==1,1], c='r', marker='o')
plt.show()

Schauen wir uns nun an, welche Parameter das Perzeptron-Modell aus *Sklearn* liefert:

In [ ]:
from sklearn.linear_model import Perceptron as SKPerceptron
clf = SKPerceptron(tol=1e-3, random_state=0)
clf.fit(X, y)
clf.coef_, clf.intercept_

Wir plotten nun die Entscheidungsgrenze mit der Funktion `plot_decision_regions` aus dem Paket `mlktend`.
Das müssen Sie ggf. zuerst installieren.

In [ ]:
%matplotlib inline
from mlxtend.plotting import plot_decision_regions

plot_decision_regions(X, y, clf=clf)
plt.show()

## Die Klassse Perzeptron

Nun wollen wir eine eigene Perzeptron Klasse entwickeln.
Die Namen der Member-Attribute und Funktionen ist an *Sklearn* angelegt.

In [ ]:
class Perceptron(object):
    """Perzeptron Klassifikator

    Member (Namen sind angelehnt an Sklearn)
    ------------
    eta0: Lernrate
    max_iter : Maximale Anzahl der Epochen beim Trainieren
    self.intercept_: Bias Parameter
    self.coef_: Modellparameter

    """

    def __init__(self, eta0=0.1, max_iter=10):
        self.eta0 = eta0
        self.max_iter = max_iter
        self.intercept_ = None
        self.coef_ = None

    def z(self, X):
        """Uebertragungsfunktion"""
        #a = self.intercept_ + np.dot(X,self.coef_)
        a = self.intercept_ + self.coef_@X.T
        return a

    def predict(self, X):
        """Forward Pass"""
        Z = self.z(X)
        # Aktivierungsfunktion
        a = (Z>=0.0)*1.0
        return a.ravel()
    
    def fit(self, X, y):
        """Fit-Funktion

        Parameter
        ----------
        X : Trainings-Datensatz
            2-dimensionales NumPy Array der Groesse [m (#Datenpunkte), n (#Attribute)]
        y : Label des Trainings-Datensatzes
            1-dimensionales NumPy Array der Groesse [m (#Datenpunkte)]
        """
        # Parameter initialisieren
        self.intercept_ = np.array([0.0])
        self.coef_ = np.array([np.zeros(X.shape[1])])

        
        for self.n_iter in range(self.max_iter): # Laufe maximale Amzahl von Iterationen
            for xi, yi in zip(X, y): # Eine Zeile aus X, einen Eintrag aus y
                dwi = self.eta0 * (yi - self.predict(xi))
                self.coef_ += dwi * xi # Broadcasting, Aller coef_ werden geupdatet
                self.intercept_ += dwi # Broadcasting, Aller intercept_ werden geupdatet
            
        return self

Nun kommen wir zu unserem Schwertlilien Datensatz.
Damit wir nur zwei Klassen unterscheiden müssen, selektieren wir die ersten 100 Datenpunkte (und verwerfen damit die letzten 50 mit der 3. Klasse)
Wir selektieren ebenfalls nur die ersten beiden Spalten der Attribute, also die Kelchblatt Länge und die Kelchblatt Breite.

In [ ]:
X = iris.data[0:100, [0,1]]
y = iris.target[:100]

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

plt.figure(2, figsize=(8, 6))
plt.clf()

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm) #, cmap=plt.cm.Set1, edgecolor='k')
plt.xlabel('Kelchblatt Laenge  [cm]') # Spalte 0
plt.ylabel('Kelchblatt Breite  [cm]') # Spalte 1
#plt.ylabel('Kronblatt Laenge  [cm]') # Spalte 2

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.show()

In [ ]:
model = Perceptron(eta0=0.1, max_iter=40)
model.fit(X, y)

In [ ]:
%matplotlib inline
from mlxtend.plotting import plot_decision_regions

plot_decision_regions(X, y, clf=model)
plt.xlabel('Kelchblatt Laenge [cm]')
plt.ylabel('Kelchblatt Breite [cm]')
plt.legend(loc='upper left')

plt.show()

**Aufgabe:** Wir sehen, dass die Entscheidungsgrenze noch nicht perfekt ist.
Wie hoch ist die *Classification Accuracy*?

In [ ]:
ca = None
# YOUR CODE HERE
raise NotImplementedError()
ca

In [ ]:
assert 'predict' in _i
assert ca == 0.53

**Aufgabe:** Wie Lauten die Modellparameter (und der Bias Parameter)?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Aufgabe:** Erweitere die Klasse `Perceptron` um folgende Funktionen:
1. Es soll eine Methode `score` geben, mit der die Vorhersage des Modells für eine Menge von Datenpunkten berechnet werden kann.
2. Die Fit-Methode soll frühzeitig abgebrochen werden, wenn die Update Terme alle (nahezu) Null sind. In dem Fall werden die Parameter nicht weiter optimiert und die Fit-Methode kann vorzeitig beendet werden. Speichern Sie die tatsächliche Anzahl der Epochen in dem Attribut `n_iter`.

In [ ]:
class Perceptron(object):
    """Perzeptron Klassifikator

    Member (Namen sind angelehnt an Sklearn)
    ------------
    eta0: Lernrate
    max_iter : Maximale Anzahl der Epochen beim Trainieren
    self.intercept_: Bias Parameter
    self.coef_: Modellparameter

    """

    def __init__(self, eta0=0.1, max_iter=10):
        self.eta0 = eta0
        self.max_iter = max_iter
        self.intercept_ = None
        self.coef_ = None

    def z(self, X):
        """Uebertragungsfunktion"""
        # = self.intercept_ + np.dot(X,self.coef_)
        a = self.intercept_ + self.coef_@X.T
        return a

    def predict(self, X):
        """Forward Pass"""
        Z = self.z(X)
        # Aktivierungsfunktion
        return (Z>=0.0)*1.0
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
s2 = 2
if s2==1:
    s2_text = "Kelchblatt Breite [cm]"
else:
    s2_text = "Kronblatt Laenge [cm]"
X = iris.data[0:100, [0,s2]]
y = iris.target[:100]

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

plt.figure(2, figsize=(8, 6))
plt.clf()

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm) #, cmap=plt.cm.Set1, edgecolor='k')
plt.xlabel('Kelchblatt Laenge [cm]') # Spalte 0
plt.ylabel(s2_text)

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.show()

In [ ]:
model = Perceptron(eta0=0.1, max_iter=1000)
model.fit(X, y), model.n_iter

In [ ]:
%matplotlib inline
from mlxtend.plotting import plot_decision_regions

plot_decision_regions(X, y, clf=model)
plt.xlabel('Kelchblatt Laenge  [cm]')
plt.ylabel(s2_text)
plt.legend(loc='upper left')

plt.show()